## Match genes in Tiana et al data to their respective proteins

In [3]:
import pandas as pd
import pickle
gene_protein = pickle.load(open('../data/gene_protein_ids.pickle','rb'))

In [4]:
# File downloaded from https://www.science.org/doi/suppl/10.1126/sciadv.abo3583/suppl_file/sciadv.abo3583_data_files_s1_to_s4.zip
df = pd.read_excel('../data/sciadv.abo3583_data_file_s1.xlsx', sheet_name='Nanog E7.5 +dox vs -dox', header=0)

In [5]:
df.head()

,ID,mgi_symbol,ave +dox,ave -dox,foldChange,logFC,P.Value,adj.P.Val,chr,start,end,strand,biotype,desc
0,ENSMUSG00000081272,Gm13509,0.0,22.359612,-3764.537589,-11.878257,3.187547e-29,3.572975e-26,2,49163675,49164982,1,pseudogene,predicted gene 13509 [Source:MGI Symbol;Acc:MG...
1,ENSMUSG00000078236,Pou3f1,0.0,15.794513,-2657.034587,-11.375601,1.049696e-10,9.109327e-09,4,124334896,124337899,1,protein_coding,"POU domain, class 3, transcription factor 1 [S..."
2,ENSMUSG00000043618,Eif5al3-ps,0.0,13.589231,-2294.151657,-11.163745,1.730782e-32,2.116431e-29,5,90561254,90561700,1,pseudogene,eukaryotic translation initiation factor 5A-li...
3,ENSMUSG00000049494,Gm12669,0.0,10.404775,-1749.919171,-10.773073,7.968524e-11,7.098319e-09,4,91472238,91473247,-1,pseudogene,predicted gene 12669 [Source:MGI Symbol;Acc:MG...
4,ENSMUSG00000057808,Gm10031,0.0,6.173581,-1042.026198,-10.025176,4.171538e-15,8.632516e-13,1,158454362,158455283,1,pseudogene,predicted pseudogene 10031 [Source:MGI Symbol;...


In [6]:
## Filter mitochondrial genes

# Gene list downloaded from ftp://ftp.broadinstitute.org/distribution/metabolic/papers/Pagliarini/MitoCarta2.0/Mouse.MitoCarta2.0.xls
mito_data = pd.read_excel('../data/Mouse.MitoCarta2.0.xls', sheet_name='A Mouse MitoCarta2.0', header=0)

In [7]:
mito_genes = []
for idx, row in mito_data.iterrows():
    if row['EnsemblGeneID'] in gene_protein:
        if row['MCARTA2_FDR'] < 0.05:
            mito_genes.append(row['EnsemblGeneID'])

In [8]:
len(mito_genes)

930

In [9]:
import re

In [10]:
gene_synonyms = pickle.load(open('../data/gene_synonyms.pickle','rb'))
gene_names = pickle.load(open('../data/gene_names.pickle','rb'))

In [11]:
ribosomal_genes = []
for gene_name, ids in gene_synonyms.items():
    if re.match('^M*RP[LS].*', gene_name):
        # print(gene_name)
        for gene_id in ids:
            ribosomal_genes.append(gene_id)
for gene_name, ids in gene_names.items():
    if re.match('^M*RP[LS].*', gene_name):
        # print(gene_name)
        for gene_id in ids:
            ribosomal_genes.append(gene_id)

In [12]:
len(ribosomal_genes)

1051

## Remove hemoglobin genes

In [17]:
hemoglobin_genes = []
for gene_name, ids in gene_synonyms.items():
    if re.match('^HB[AB].*', gene_name):
        # print(gene_name)
        for gene_id in ids:
            hemoglobin_genes.append(gene_id)
            print(gene_name)

HBA-3PS
HBA-PS3
HBB-BH3
HBB-BH2
HBB-BH1
HBB-BH0
HBB-Y
HBA-X
HBA-A1
HBA1
HBA-A2
HBA-4PS
HBA-PS4
HBA-A4
HBB-BT
HBB-BS


In [23]:
hemoglobin_genes

['ENSMUSG00000114969',
 'ENSMUSG00000114969',
 'ENSMUSG00000083216',
 'ENSMUSG00000078621',
 'ENSMUSG00000052217',
 'ENSMUSG00000085700',
 'ENSMUSG00000052187',
 'ENSMUSG00000055609',
 'ENSMUSG00000069919',
 'ENSMUSG00000069919',
 'ENSMUSG00000069917',
 'ENSMUSG00000084893',
 'ENSMUSG00000084893',
 'ENSMUSG00000084893',
 'ENSMUSG00000073940',
 'ENSMUSG00000052305']

In [18]:
fold_changes = {}
not_found = {}

# For each row in the data frame
for idx,row in df.iterrows():
    ensembl_id = row['ID']
    fold_change = float(row['foldChange'])
    gene_type = row['biotype']
    chromosome = row['chr']
    
    # Only include protein coding genes
    protein_match = ensembl_id in gene_protein
    # Exclude sex chromosome and mitochondrial genes
    chromosomes = chromosome not in ['X', 'Y', 'MT']
    not_mito = ensembl_id not in mito_genes
    # Exclude ribosomal genes
    not_ribosomal = ensembl_id not in ribosomal_genes  
    # Exclude hemoglobin genes
    not_hemoglobin = ensembl_id not in hemoglobin_genes

    if protein_match and chromosomes and not_mito and not_ribosomal and not_hemoglobin:
        protein_id = gene_protein[ensembl_id]
        fold_changes[protein_id] = fold_change
    if not protein_match:
        not_found[ensembl_id] = gene_type

from collections import Counter
gene_type_count = Counter()
# Count of gene types of the unmatched genes
for ensembl_id, gene_type in not_found.items():
    gene_type_count[gene_type] += 1
print(gene_type_count.most_common())

# # Print the ensemble ids of the unmatched protein coding genes
# for ensembl_id, gene_type in not_found.items():
#     if gene_type == 'protein_coding':
#         print(ensembl_id, gene_type)

[('pseudogene', 61), ('protein_coding', 31), ('lincRNA', 28), ('processed_transcript', 23), ('Mt_tRNA', 3), ('Mt_rRNA', 2), ('miRNA', 1)]


In [19]:
from collections import Counter

In [20]:
Counter([fc > 0 for fc in fold_changes.values()]).most_common()

[(False, 700), (True, 629)]

In [21]:
import pickle

In [22]:
with open('../data/tiana_etal_differential_expression.csv','w') as outputfile:
    for gene, fc in fold_changes.items():
        outputfile.write(','.join([gene,str(fc > 0), str(fc)]) + '\n')
